<a href="https://colab.research.google.com/github/mvish7/PyTorch/blob/master/NN_in_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

necessary imports

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

fun to calculate number of correct preds

In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

creating n/w class and building the fwd method

In [0]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    
  def forward(self,t):
    #implementing fwd pass
    
    # input layer
    t=t #returning the tensor as it as, no need of processing
    
    # hidden layer 1 
    t=self.conv1(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    
    #hidden layer 2
    t=self.conv2(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden layer 3
    t=t.reshape(-1,12*4*4)  #doing the flattening operation before passing the tensor from conv layer to the linear layer
    t=self.fc1(t)
    t=F.relu(t)
    
    #hidden layer 4
    t=self.fc2(t)
    t=F.relu(t)
    
    #output layer
    t=self.out(t)
    #t=F.softmax(t, dim=1)  no need to use softmax here, coz we are using cross entropy as our loss function and it has built in softmax.
    
    
    return t
  
      

data downloaded

In [0]:
trainset=torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',
                                           train=True, 
                                           download=True,
               transform=transforms.Compose([transforms.ToTensor()]))

downloading the test data now

In [0]:
testset = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=False,
                                       download=True, 
                                           transform=transforms.Compose([transforms.ToTensor()]))

In [0]:
network=Network()

creating batch of 100 images, using dataloader class

In [0]:
data_loader=torch.utils.data.DataLoader(trainset,batch_size=100, shuffle='true')
batch=next(iter(data_loader))
images, labels=batch

creating dataloader instance for test data

In [0]:
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False)

calculating the loss

In [0]:
preds= network(images)
loss_fun= nn.CrossEntropyLoss()
#loss.item() #item method gives out the numerical vlaue of the loss

calculating the gradients with backward method

In [0]:
loss=loss_fun(preds,labels)
loss.backward()

now using this gradients to update the network weights using optimizer

In [0]:
optimizer=optim.Adam(network.parameters(), lr=0.01)

In [27]:
loss.item() #just checking the loss once again

2.3104960918426514

In [28]:
get_num_correct(preds, labels) # we have 12/100 correct

7

now updating the weights using step() method

In [0]:
optimizer.step()

now let's do this steps once again and see of n/w actially learns from the gradients we calculated and updated

In [30]:
preds= network(images)
loss= F.cross_entropy(preds, labels)
loss.item()

2.2861897945404053

In [31]:
get_num_correct(preds, labels)

12

yayy! we got reduced loss and more correct predictions

now creating the training loop

In [0]:
images.shape

setting the training loop

In [34]:
for epoch in range(5):
  total_loss=0
  total_correct=0

  for batch in data_loader:
    images, labels= batch
    preds = network(images)
    loss = loss_fun(preds, labels)
  
    optimizer.zero_grad()
    loss.backward() #calculating the gradients
    optimizer.step() #update weights
  
    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
  
  print("epoch:", epoch, "total correct:", total_correct, "loss:", total_loss)

epoch: 0 total correct: 52589 loss: 200.86878828704357
epoch: 1 total correct: 52781 loss: 194.71054181456566
epoch: 2 total correct: 52995 loss: 190.26656316965818
epoch: 3 total correct: 53088 loss: 186.94299352169037
epoch: 4 total correct: 53344 loss: 178.01390904188156


In [36]:
total_correct/len(trainset)  #is the accuracy

0.8890666666666667

In [0]:
test_imgs,test_label=next(iter(testloader))
preds=network(test_imgs)
get_num_correct(preds,test_label)

so 40/50 were predicted correctly

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        preds = network(images)
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))